# This notebook is for testing purposes , It is not the final deliverable.
## Contents:
1. [Diffie-Hellman Key Exchange](#1)
2. [AlGamal](#2)
3. [Verifying Signature](#3)
4. [Handling Messages with AES](#4)
5. [Integration to chat.py](#5)

### Import relevant libraries

In [13]:
from helpers.dh_gamal import deffie_hellman
from helpers.tools import from_file
import numpy as np
import hashlib

%reload_ext autoreload
%autoreload 2

ImportError: cannot import name 'deffie_hellman' from 'helpers.dh_gamal' (c:\Users\Oreo\Documents\Security\Project\helpers\dh_gamal.py)

### 1. [Diffie-Hellman Key Exchange](#1) (Done)

In [ ]:
q_dh , a_dh , q_gamal , a_gamal = from_file("agree.txt")

In [ ]:
X_a = np.random.randint(2, q_dh)
print("Alice Private Key for DH: ", X_a)
X_b = np.random.randint(2, q_dh)
print("Bob Private Key for DH: ", X_b)
Y_a = deffie_hellman(prime=q_dh, generator=a_dh, private_key=X_a)
print("Alice Public Key for DH: ", Y_a)
Y_b = deffie_hellman(prime=q_dh, generator=a_dh, private_key=X_b)
print("Bob Public Key for DH: ", Y_b)



Alice Private Key for DH:  301
Bob Private Key for DH:  125
Alice Public Key for DH:  249
Bob Public Key for DH:  104


In [ ]:
X_a2 = np.random.randint(2,q_gamal)
print("Alice Private Key for Al-Gamal: ", X_a2)
X_b2 = np.random.randint(2, q_gamal)
print("Bob Private Key for Al-Gamal: ", X_b2)
Y_a2 = deffie_hellman(prime=q_gamal, generator=a_gamal, private_key=X_a2)
print("Alice Public Key for Al-Gamal: ", Y_a2)
Y_b2 = deffie_hellman(prime=q_gamal, generator=a_gamal, private_key=X_b2)
print("Bob Public Key for Al-Gamal: ", Y_b2)

Alice Private Key for Al-Gamal:  229
Bob Private Key for Al-Gamal:  60
Alice Public Key for Al-Gamal:  51
Bob Public Key for Al-Gamal:  11


3la taree2et el document ( Not solvable and doesn't verify correctly)

In [15]:
def gcd(num1, num2):
    assert num1 > num2, "First Number should be greater"
    _, r = divmod(num1, num2)
    return num2 if r == 0 else gcd(num2, r)

def generate_random_k(prime):
    k = np.random.randint(2,prime-2)
    while gcd(prime-1,k) != 1 :
        k = np.random.randint(2,prime-2)
    return k

def extended_euclidean(a,b):
    if a == 0:
        return b, 0, 1
    else:
        g, x, y = extended_euclidean(b % a, a)
        return g, y - (b // a) * x, x
    

def send_signature_sha1(public_dh,private_gamal,generator,prime):
    m = hashlib.sha1(str(public_dh).encode()).hexdigest()
    # take last 8 bits 
    m = int(m[-1:],16)
    assert 0 < m < prime , "m doesn't satisfy the condition 0 < m < prime"
    # generate random K 
    
    r,s = 0,0 # in the unlikely event of s = 0 , you will have to regenerate the k
    while s == 0 :
        k = generate_random_k(prime)    
        r = pow(generator,k,prime)
        s = (m - private_gamal*r) * extended_euclidean(k,prime-1)[1] % (prime-1)
    print(f"m: {m}, r: {r}, s: {s} , k: {k} ")
    return r, s

def verify_signature_sha1(public_dh,public_gamal,private_gamal,generator,prime,r,s):
    assert 0 < r < prime , "r not in permissible range"
    assert 0 < s < prime - 1 , "s not in permissble range"
    # to verify then 
    # g**m = (public_dh**r * r**s) mod prime
    m = hashlib.sha1(str(public_dh).encode()).hexdigest()
    m = int(m[-1:],16)
    left = pow(generator,m,prime)
    print(f"left : {left}")
    right = (pow(public_gamal,r) * pow(r,s)) % prime
    print(f"right = {right}")
    print(f"m: {m}, r: {r}, s: {s} ")
    if left == right:
        key = hashlib.sha256(str(pow(public_dh,private_gamal,prime)).encode()).hexdigest()
        return key.encode()
    else:
        print("Signature is not valid , Ending Connection ...")
        return None


In [16]:
# send signature sha1
r , s= send_signature_sha1(Y_a,X_a2,a_gamal,q_gamal)
print("Signature: ", r,s)



m: 9, r: 325, s: 336 , k: 251 
Signature:  325 336


In [17]:
#verify signature sha1
verify_signature_sha1(Y_a,Y_a2,a_gamal,q_gamal,r,s)

left : 268
right = 268
m: 9, r: 325, s: 336 


b'8df66f64b57424391d363fd6b811fed3c430c77597da265025728bd637bad804'

In [ ]:
print(Y_a2,Y_b2)

### My OWN SHA1